In [ ]:
# 0: copy sir encoding
# 59:53.45 to get an encoding (needing debugging/checking)
# 01:41:13 configure parameter bounds to working test case
# 03:00:00 debug model 
# 03:00:00 test solvers and extend compartmental bounds with noise
# +33:37.29 fix step size handling in encoding
# +1:00:00 debug step size and plotting

from pysmt.shortcuts import And, Or, Plus, Minus, Times, Div, REAL, LE, LT, GE, GT, Equals, Symbol, Real, Solver, TRUE, FALSE
from pysmt.fnode import  FNode
from typing import Dict
import pandas as pd
from decimal import Decimal
import numpy as np


def dataframe(assignment: Dict[Symbol, float], state_variables, parameters, max_step, step_size) -> pd.DataFrame:
    timepoints = list(range(0, max_time+step_size, step_size))
    timeseries = {sv: [None]*len(timepoints) for sv in state_variables}
    for k, v in assignment.items():
        sym = k.symbol_name()
        if "_" in sym:
            sv = sym.split("_")[0]
            t = sym.split("_")[1]
            value =Decimal(v.numerator) / Decimal(v.denominator)
            if max_time > 0:
                timepoint = timepoints.index(int(t))
                timeseries[sv][timepoint] = value
            else:
                timeseries[sv][timepoints.index(0)] = value
        else:
            timeseries[sym] = [v]*len(timepoints)
    df = pd.DataFrame(timeseries, index=timepoints).astype(float)
    return df

def reindex_and_interpolate(df, new_index):
    df_reindexed = df.reindex(index = new_index)
    df_reindexed.interpolate(method = 'linear', inplace = True)
    return df_reindexed


def plot_results(values, max_step, step_size):
    timepoints = list(range(0, max_time+step_size, step_size))
    results: pd.DataFrame = dataframe(values, ["Susceptible", "Diagnosed", "Infected", "Ailing", "Recognized", "Healed", "Threatened", "Extinct"], ["beta", "gamma", "delta", "alpha", "epsilon", "zeta", "lambdax", "eta", "rho", "theta", "kappa", "mu", "nu", "xi", "tau", "sigma"], max_step, step_size)
    newindex = np.linspace(timepoints[0], timepoints[-1], timepoints[-1]+1)
    results = reindex_and_interpolate(results, newindex)
    print(f"beta = {results['beta'][0]}, lambda = {results['lambdax'][0]}")
    print(results[["Susceptible", "Diagnosed", "Infected", "Ailing", "Recognized", "Healed", "Threatened", "Extinct"]])
    ax = results[["Susceptible", "Diagnosed", "Infected", "Ailing", "Recognized", "Healed", "Threatened", "Extinct"]].plot()
    ax.set_xlabel="Time"
    return ax, results

def run_solver(solver, formula):
    solver.add_assertion(formula)
    result = solver.solve()
    if result:
        model = solver.get_model()
        variables = formula.get_free_variables()
        values = {}
        for var in variables:
            try:
                values[var] = model.get_value(var).constant_value()
            except Exception as e:
                pass
    else:
        print("Unsat")
        values = None
    return values



In [ ]:
# from funman.api.run import Runner
# import json

# amr_model = "../../resources/amr/petrinet/mira/models/BIOMD0000000955_askenet.json"
# m = Runner().get_model(amr_model)
# m[0].to_dot()

In [ ]:
# Configure these bounds (lower, upper) to configure the parameter space

noise = 1e-1

beta_bounds = (0.2,  0.8)
# gamma_bounds = (0.3648, 0.5472)
# alpha_bounds = ( 0.45599999999999996,0.6839999999999999)
# delta_bounds = (0.008799999999999999, 0.0132)
# epsilon_bounds = (0.1368, 0.20520000000000002)
# zeta_bounds = (0.1, 0.15)
lambdax_bounds = (0.0, 0.1)
# eta_bounds = (0.1,  0.15)
# rho_bounds = (0.027200000000000002, 0.0408)
# theta_bounds = (0.2968, 0.4452)
# kappa_bounds = (0.013600000000000001, 0.0204)
# mu_bounds = (0.013600000000000001, 0.0204)
# nu_bounds = (0.0216, 0.0324)
# xi_bounds = (0.013600000000000001, 0.0204)
# tau_bounds = (0.008, 0.012)
# sigma_bounds = (0.013600000000000001, 0.0204)

# {"alpha":0.570, "beta":0.011, "delta":0.011, "gamma":0.456, "epsilon":0.171, "theta":0.371, "eta":0.125, "zeta":0.125,"mu":0.017, "nu":0.027,"tau":0.01, "lambda":0.034, "rho":0.034, "kappa":0.017, "xi":0.017,"sigma":0.017, "days": 1}

param_noise = 0
# beta_bounds = (0.011-param_noise,  0.011+param_noise)
gamma_bounds = (0.0456-param_noise, 0.0456+param_noise)
alpha_bounds = ( 0.1-param_noise, 0.1+param_noise)
delta_bounds = (0.00011-param_noise, 0.00011+param_noise)
epsilon_bounds = (0.03-param_noise, 0.03+param_noise)
zeta_bounds = (0.0125-param_noise, 0.0125+param_noise)
# lambdax_bounds = (0.0-param_noise, 0.1+param_noise)
eta_bounds = (0.0125-param_noise, 0.0125+param_noise)
rho_bounds = (0.034-param_noise, 0.034+param_noise)
theta_bounds = (0.571-param_noise, 0.571+param_noise) #(epsilon_bounds[0]*2.0, epsilon_bounds[1]*2.0) #(0.2968, 0.4452)
kappa_bounds = (0.017-param_noise, 0.017+param_noise)
mu_bounds = (0.017-param_noise, 0.017+param_noise)
nu_bounds = (0.027-param_noise, 0.027+param_noise)
xi_bounds = (0.017-param_noise, 0.017+param_noise)
tau_bounds = (0.01-param_noise, 0.01+param_noise)
sigma_bounds = (0.017-param_noise, 0.017+param_noise)


# Set these values for the initial state

Susceptible_0_value = 0.99
Diagnosed_0_value = 0.01
Infected_0_value = 0.0
Ailing_0_value = 0.0
Recognized_0_value = 0.0
Healed_0_value = 0
Threatened_0_value = 0
Extinct_0_value = 0


# Timepoints
step_size = 2
max_time = 10

# 11:34 - adapting names of variables and parameters



In [ ]:
def encode(step_size=1, max_time=10, noise=1e-3, strict_upper_bound_parameters = False, use_compartmental=True):
    # The main encoding code, which includes 

    time_format = lambda t: f"{t:03d}"

    timepoints = list(range(0, max_time+step_size, step_size))

    ################################################################################
    ################# Initial States ###############################################
    ################################################################################

    Susceptible_0 = Symbol(f"Susceptible_{time_format(0)}", REAL)
    Diagnosed_0 = Symbol(f"Diagnosed_{time_format(0)}", REAL)
    Infected_0 = Symbol(f"Infected_{time_format(0)}", REAL)
    Ailing_0 = Symbol(f"Ailing_{time_format(0)}", REAL)
    Recognized_0 = Symbol(f"Recognized_{time_format(0)}", REAL)
    Healed_0 = Symbol(f"Healed_{time_format(0)}", REAL)
    Threatened_0 = Symbol(f"Threatened_{time_format(0)}", REAL)
    Extinct_0 = Symbol(f"Extinct_{time_format(0)}", REAL)

    population_size = Susceptible_0_value + Diagnosed_0_value + Infected_0_value + Ailing_0_value + Recognized_0_value + Healed_0_value + Threatened_0_value + Extinct_0_value

    # SIR Model Initial State
    initial_state = And([
        Equals(Susceptible_0, Real(Susceptible_0_value)),
        Equals(Infected_0, Real(Infected_0_value)),
        Equals(Diagnosed_0, Real(Diagnosed_0_value)),
        Equals(Ailing_0, Real(Ailing_0_value)),
        Equals(Recognized_0, Real(Recognized_0_value)),
        Equals(Threatened_0, Real(Threatened_0_value)),
        Equals(Healed_0, Real(Healed_0_value)),
        Equals(Extinct_0, Real(Extinct_0_value))
    ])

    ################################################################################
    ################# Parameters     ###############################################
    ################################################################################


    # Parameters
    beta = Symbol("beta", REAL)
    gamma = Symbol("gamma", REAL)
    delta = Symbol("delta", REAL)
    alpha = Symbol("alpha", REAL)
    epsilon = Symbol("epsilon", REAL)
    zeta = Symbol("zeta", REAL)
    lambdax = Symbol("lambdax", REAL)
    eta = Symbol("eta", REAL)
    rho = Symbol("rho", REAL)
    theta = Symbol("theta", REAL)
    kappa = Symbol("kappa", REAL)
    mu = Symbol("mu", REAL)
    nu = Symbol("nu", REAL)
    xi = Symbol("xi", REAL)
    tau = Symbol("tau", REAL)
    sigma = Symbol("sigma", REAL)


    upper_op = LT if strict_upper_bound_parameters else LE
    parameters = And([
        And(LE(Real(beta_bounds[0]), beta), upper_op(beta, Real(beta_bounds[1]))),
        And(LE(Real(gamma_bounds[0]), gamma), upper_op(gamma, Real(gamma_bounds[1]))),
        And(LE(Real(delta_bounds[0]), delta), upper_op(delta, Real(delta_bounds[1]))),
        And(LE(Real(alpha_bounds[0]), alpha), upper_op(alpha, Real(alpha_bounds[1]))),
        And(LE(Real(epsilon_bounds[0]), epsilon), upper_op(epsilon, Real(epsilon_bounds[1]))),
        And(LE(Real(zeta_bounds[0]), zeta), upper_op(zeta, Real(zeta_bounds[1]))),
        And(LE(Real(lambdax_bounds[0]), lambdax), upper_op(lambdax, Real(lambdax_bounds[1]))),
        And(LE(Real(eta_bounds[0]), eta), upper_op(eta, Real(eta_bounds[1]))),
        And(LE(Real(rho_bounds[0]), rho), upper_op(rho, Real(rho_bounds[1]))),
        And(LE(Real(kappa_bounds[0]), kappa), upper_op(kappa, Real(kappa_bounds[1]))),
        And(LE(Real(theta_bounds[0]), theta), upper_op(theta, Real(theta_bounds[1]))),
        And(LE(Real(mu_bounds[0]), mu), upper_op(mu, Real(mu_bounds[1]))),
        And(LE(Real(nu_bounds[0]), nu), upper_op(nu, Real(nu_bounds[1]))),
        And(LE(Real(xi_bounds[0]), xi), upper_op(xi, Real(xi_bounds[1]))),
        And(LE(Real(tau_bounds[0]), tau), upper_op(tau, Real(tau_bounds[1]))),
        And(LE(Real(sigma_bounds[0]), sigma), upper_op(sigma, Real(sigma_bounds[1])))
    ])




    ################################################################################
    ################# Transitions ##################################################
    ################################################################################

    Susceptible_next = lambda t: Symbol(f"Susceptible_{time_format(t+step_size)}", REAL)
    Susceptible_now = lambda t: Symbol(f"Susceptible_{time_format(t)}", REAL)
    Diagnosed_next = lambda t: Symbol(f"Diagnosed_{time_format(t+step_size)}", REAL)
    Diagnosed_now = lambda t: Symbol(f"Diagnosed_{time_format(t)}", REAL)
    Infected_next = lambda t: Symbol(f"Infected_{time_format(t+step_size)}", REAL)
    Infected_now = lambda t: Symbol(f"Infected_{time_format(t)}", REAL)
    Ailing_next = lambda t: Symbol(f"Ailing_{time_format(t+step_size)}", REAL)
    Ailing_now = lambda t: Symbol(f"Ailing_{time_format(t)}", REAL)
    Healed_next = lambda t: Symbol(f"Healed_{time_format(t+step_size)}", REAL)
    Healed_now = lambda t: Symbol(f"Healed_{time_format(t)}", REAL)
    Threatened_next = lambda t: Symbol(f"Threatened_{time_format(t+step_size)}", REAL)
    Threatened_now = lambda t: Symbol(f"Threatened_{time_format(t)}", REAL)
    Recognized_next = lambda t: Symbol(f"Recognized_{time_format(t+step_size)}", REAL)
    Recognized_now = lambda t: Symbol(f"Recognized_{time_format(t)}", REAL)
    Extinct_next = lambda t: Symbol(f"Extinct_{time_format(t+step_size)}", REAL)
    Extinct_now = lambda t: Symbol(f"Extinct_{time_format(t)}", REAL)
    dt = Real(float(step_size))


    # t1: D, S -> D, I  Diagnosed*Susceptible*beta
    # t2: A, S -> A, I  Ailing*Susceptible*gamma
    # t3: R, S -> R, I  Recognized*Susceptible*delta
    # t4: I, S -> I, I  Infected*Susceptible*alpha
    # t5: I -> D    Infected*epsilon
    # t6: I -> A    Infected*zeta
    # t7: I -> H    Infected*lambdax
    # t8: D -> R    Diagnosed*eta
    # t9: D -> H    Diagnosed*rho"
    # t10: A -> R   Ailing*theta
    # t11: A -> H   Ailing*kappa
    # t12: A -> T   Ailing*mu
    # t13: R -> T   Recognized*nu
    # t14: R -> H   Recognized*xi
    # t15: T -> E   Threatened*tau
    # t16: T -> H   Threatened*sigma

    # -t1 -t2 -t3 -t4
    Susceptible_Trans = lambda t: Equals(Susceptible_next(t), 
                                    Plus(
                                        Susceptible_now(t), 
                                        Times(Plus([
                                            Times(Real(-1.0), Diagnosed_now(t), Susceptible_now(t), beta), 
                                            Times(Real(-1.0), Ailing_now(t), Susceptible_now(t), gamma),
                                            Times(Real(-1.0), Recognized_now(t), Susceptible_now(t), delta),
                                            Times(Real(-1.0), Infected_now(t), Susceptible_now(t), alpha)]), 
                                            dt)))

    # -t1+t1 -t8 -t9 + t5
    Diagnosed_Trans = lambda t: Equals(Diagnosed_next(t), 
                                    Plus(
                                        Diagnosed_now(t), 
                                        Times(Plus([
                                            Times(Real(-1.0), Diagnosed_now(t), eta), 
                                            Times(Real(-1.0), Diagnosed_now(t), rho), 
                                            Times( Infected_now(t), epsilon)]),
                                            dt)))

    # +t1 +t2 +t3 +t4 -t5 -t6 -t7 
    Infected_Trans = lambda t: Equals(Infected_next(t), 
                                    Plus(
                                        Infected_now(t), 
                                        Times(Plus([
                                            Times(Diagnosed_now(t), Susceptible_now(t), beta), # t1
                                            Times(Ailing_now(t), Susceptible_now(t), gamma), # t2
                                            Times(Recognized_now(t), Susceptible_now(t), delta), # t3
                                            Times(Infected_now(t), Susceptible_now(t), alpha), # t4
                                            Times(Real(-1.0), Infected_now(t), epsilon), # t5
                                            Times(Real(-1.0), Infected_now(t), zeta), # t6
                                            Times(Real(-1.0), Infected_now(t), lambdax)]), #t7
                                            dt)))

    # -t2 +t2 +t6 -t10 -t11 -t12
    Ailing_Trans = lambda t: Equals(Ailing_next(t), 
                                    Plus(
                                        Ailing_now(t), 
                                        Times(Plus([
                                            Times(Infected_now(t), zeta), 
                                            Times(Real(-1.0), Ailing_now(t), theta),
                                            Times(Real(-1.0), Ailing_now(t), kappa),
                                            Times(Real(-1.0), Ailing_now(t), mu)]),
                                            dt)))

    # +t3 -t3 +t8 +t10 -t13 -t14 
    Recognized_Trans = lambda t: Equals(Recognized_next(t), 
                                    Plus(
                                        Recognized_now(t), 
                                        Times(Plus([
                                            Times(Diagnosed_now(t), eta), 
                                            Times(Ailing_now(t), theta),
                                            Times(Real(-1.0), Recognized_now(t), nu),
                                            Times(Real(-1.0), Threatened_now(t), xi)]),
                                            dt)))

    # +t7 +t9 +t11 +t14 +t16
    Healed_Trans = lambda t: Equals(Healed_next(t), 
                                    Plus(
                                        Healed_now(t), 
                                        Times(Plus([
                                            Times(Infected_now(t), lambdax), 
                                            Times(Diagnosed_now(t), rho),
                                            Times(Ailing_now(t), kappa),
                                            Times(Recognized_now(t), xi),
                                            Times(Threatened_now(t), sigma)]),
                                            dt)))

    # +t12 +t13 -t15 -t16
    Threatened_Trans = lambda t: Equals(Threatened_next(t), 
                                    Plus(
                                        Threatened_now(t), 
                                        Times(Plus([
                                            Times(Ailing_now(t), mu),
                                            Times(Recognized_now(t), nu),
                                            Times(Real(-1.0), Threatened_now(t), tau),
                                            Times(Real(-1.0), Threatened_now(t), sigma)]),
                                            dt)))

    # -t15
    Extinct_Trans = lambda t: Equals(Extinct_next(t), 
                                    Plus(
                                        Extinct_now(t), 
                                        Times(Plus([
                                            Times(Threatened_now(t), tau)]),
                                            dt)))


    Trans = lambda t: And([
        Susceptible_Trans(t), Diagnosed_Trans(t), 
        Infected_Trans(t), Ailing_Trans(t), Recognized_Trans(t), 
        Healed_Trans(t), Threatened_Trans(t), Extinct_Trans(t)
        ])

    All_Trans = And([Trans(t) for t in timepoints[:-1]])


    ################################################################################
    ################# Constraints ##################################################
    ################################################################################

    pop_sum = lambda t: Plus([
            Susceptible_now(t), Diagnosed_now(t), Infected_now(t),
            Ailing_now(t), Recognized_now(t), Healed_now(t),
            Threatened_now(t), Extinct_now(t)
        ])

    compartmental_constraint = And([
        And( 
        LE(Real(0.0), Susceptible_now(t)),
        LE(Real(0.0), Diagnosed_now(t)),
        LE(Real(0.0), Infected_now(t)),
        LE(Real(0.0), Ailing_now(t)),
        LE(Real(0.0), Recognized_now(t)),
        LE(Real(0.0), Healed_now(t)),
        LE(Real(0.0), Threatened_now(t)),
        LE(Real(0.0), Extinct_now(t)),
        LE(pop_sum(t), Real(population_size+noise)),
        LE(Real(population_size-noise), pop_sum(t))
        )
    for t in timepoints]) if use_compartmental else TRUE()

    # 10m to add and check
    I_peak = (0.45, 0.55) # .45, .55
    I_peak_t = (45.0, 55.0) # 45, 55
    peak_I = And([
        And(LE(Real(I_peak[0]), Infected_now(t)),
        LT(Infected_now(t), Real(I_peak_t[1])))
        for t in timepoints 
        if I_peak_t[0] <= t and t <= I_peak_t[1]
    ])


    ################################################################################
    ################# Combine Assertions ###########################################
    ################################################################################

    consistency = And([
        initial_state,
        parameters,
        All_Trans,
       compartmental_constraint,
        peak_I
        ])
    # print(consistency.serialize())
    return consistency

In [ ]:
# encode(max_time=max_time, step_size=step_size, noise=noise, use_compartmental=False).serialize(

# )

In [ ]:
# Use dreal instead of z3
from pysmt.logics import QF_NRA
from funman_dreal.solver import ensure_dreal_in_pysmt

ensure_dreal_in_pysmt()

opts = {
        "dreal_precision": 1e-3,
        "dreal_log_level": "info",
        "dreal_mcts": True,
    }

max_time=20
step_size=1
noise = 0.0001

with Solver(name="dreal",
            logic=QF_NRA,
            solver_options=opts
            ) as solver:
    values = run_solver(solver, encode(max_time=max_time, step_size=step_size, noise=noise, use_compartmental=False))
    
if values:
    ax, results = plot_results(values, max_time, step_size)
    f_pop = results.loc[float(max_time)][["Susceptible", "Diagnosed", "Infected", "Ailing", "Recognized", "Healed", "Threatened", "Extinct"  ]].sum()
    print(f"Population at end: {f_pop}")
    # results.loc[10.0]


In [ ]:
# max_time=20
# step_size=1



# # Solve encoding
# with Solver() as solver:
#     values = run_solver(solver, encode(max_time=max_time, step_size=step_size, noise=noise))
    
# if values:
#     plot_results(values, max_time, step_size)
# values